In [143]:
import os
import numpy as np
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 
import re

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [145]:
import nltk 
from nltk import ngrams
from nltk.text import Text 
nltk.download("stopwords") 
#--------# 
from nltk.corpus import stopwords 

#Create lemmatizer and stopwords list 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sony\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [146]:
episodes = []
for root, dirs, files in os.walk('friends'):
    for name in files:
        episodes.append(os.path.join(root, name))

In [147]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text) 
    text = re.sub(r'\d', '', text) 
    text = re.sub(r'[A-Za-z]', '', text) 
    text = [pmm.normal_forms(x)[0] for x in text.split() if x not in russian_stopwords] 
    return text

In [148]:
texts = {}
for i in episodes:
    with open(i, 'r', encoding='utf-8') as t:
        text = t.read()
        new_text = clean_text(text)
        texts[i] = new_text

In [151]:
##Получение названий серий
titles = []
for root, dirs, files in os.walk('friends'):
    for name in files:
        titles.append(os.path.join(name))

In [149]:
corpus = []
for key, value in texts.items():
    x = ' '.join(value)
    corpus.append(x)

In [202]:
X = vectorizer.fit_transform(corpus)
f_matrix = X.toarray()

In [203]:
## Создание обратной матрицы
matrix = np.transpose(f_matrix)
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [204]:
len(matrix)

15240

In [227]:
query = input('Введите запрос: ')

Введите запрос: хочу


In [228]:
norm_query = clean_text(query)
norm_query

['хотеть']

In [229]:
##Создание матрицы для запроса
m_query = []
for i in vectorizer.get_feature_names():
    if i in query:
        m_query.append(1)
    else:
        m_query.append(0)

In [232]:
##Поиск запроса в текстах (почему-то не работает так, как надо, а почему - я не смогла разобраться...)
response = {}
for index, item in enumerate(titles):
    k = 0
    column = matrix[:,index]
    for a,b in zip(m_query, column):
        if b >= 0 and a == 1:
            k += 1
    if k > 0:
        response[item] = k

In [238]:
##Выдает документы, в которых встретились слова из запроса. 
##Чем больше в документе содержится слов из запроса, тем выше он в выдаче (по идее работает, но из-за сломанного поиска этого не видно)
sorted_resp = sorted(response.items(), key=lambda x: x[1], reverse=True)
for i in sorted_resp:
    print(i[0])

Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt
Friends - 1x02 - The One With The Sonogram At The End.ru.txt
Friends - 1x03 - The One With The Thumb.ru.txt
Friends - 1x04 - The One With George Stephanopoulos.ru.txt
Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt
Friends - 1x06 - The One With The Butt.ru.txt
Friends - 1x07 - The One With The Blackout.ru.txt
Friends - 1x08 - The One Where Nana Dies Twice.ru.txt
Friends - 1x09 - The One Where Underdog Gets Away.ru.txt
Friends - 1x10 - The One With The Monkey.ru.txt
Friends - 1x11 - The One With Mrs. Bing.ru.txt
Friends - 1x12 - The One With The Dozen Lasagnas.ru.txt
Friends - 1x13 - The One With The Boobies.ru.txt
Friends - 1x14 - The One With The Candy Hearts.ru.txt
Friends - 1x15 - The One With The Stoned Guy.ru.txt
Friends - 1x16 - The One With Two Parts (1).ru.txt
Friends - 1x17 - The One With Two Parts (2).ru.txt
Friends - 1x18 - The One With All The Poker.ru.txt
Friends - 1x19 - The One Where The 

In [43]:
## a) b) Поиск самого частотного и самого редкого слов в сериях
sums = []
for row in matrix:
    sums.append(sum(row))
    
max_index = sums.index(max(sums))
min_index = sums.index(min(sums))
words = vectorizer.get_feature_names()
print('Самое частотное слово в сериях: "' + words[max_index] + '"')
print('Самое редкое слово в сериях: "' + words[min_index] + '"')

Самое частотное слово в сериях: "это"
Самое редкое слово в сериях: "аааа"


In [136]:
## c) Набор слов во всех документах коллекции
in_all_docs = []
for row, word in zip(matrix, vectorizer.get_feature_names()):
    if 0 not in row:
        in_all_docs.append(word)
print('Слова, встречающиеся во всех документах:', ', '.join(in_all_docs) + '.')

Слова, встречающиеся во всех документах: весь, да, думать, знать, как, мочь, не, нет, но, просто, ты, хотеть, что, это.
